https://github.com/R3gm/SoniTranslate/tree/main

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title Mount Google Drive for larger Video Input (It will create a folder Inside Google Drive called ```upload```. Then Upload your Video and Image in this folder.)
from IPython.display import clear_output
%cd /content/
from google.colab import drive
import os
import shutil
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/
!mkdir upload
clear_output()

In [ ]:
#@title Install and Restart
%cd /content/
!rm -rf /content/whisperX
!git clone https://github.com/m-bain/whisperX.git
%cd /content/whisperX
!pip install .
!echo /usr/lib64-nvidia/ >/etc/ld.so.conf.d/libcuda.conf; ldconfig
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64:/usr/local/cuda/lib
!pip install srt
!pip install deep_translator==1.11.4
!pip install edge_tts==6.1.7
!pip install gradio==4.19.2
!pip install pydub==0.25.1
!pip install pysrt==1.1.2
!pip install librosa==0.10.2.post1
from IPython.display import clear_output
clear_output()
import time
time.sleep(5)
import os
os.kill(os.getpid(), 9)

In [17]:
#@title whisper config
import whisperx
import gc
import torch
from whisperx.alignment import (
    DEFAULT_ALIGN_MODELS_TORCH as DAMT,
    DEFAULT_ALIGN_MODELS_HF as DAMHF,
)

LANGUAGES_UNIDIRECTIONAL = {
    "Aymara (ay)": "ay",
    "Bambara (bm)": "bm",
    "Cebuano (ceb)": "ceb",
    "Chichewa (ny)": "ny",
    "Divehi (dv)": "dv",
    "Dogri (doi)": "doi",
    "Ewe (ee)": "ee",
    "Guarani (gn)": "gn",
    "Iloko (ilo)": "ilo",
    "Kinyarwanda (rw)": "rw",
    "Krio (kri)": "kri",
    "Kurdish (ku)": "ku",
    "Kirghiz (ky)": "ky",
    "Ganda (lg)": "lg",
    "Maithili (mai)": "mai",
    "Oriya (or)": "or",
    "Oromo (om)": "om",
    "Quechua (qu)": "qu",
    "Samoan (sm)": "sm",
    "Tigrinya (ti)": "ti",
    "Tsonga (ts)": "ts",
    "Akan (ak)": "ak",
    "Uighur (ug)": "ug"
}
LANGUAGES = {
    "Automatic detection": "Automatic detection",
    "Arabic (ar)": "ar",
    "Chinese - Simplified (zh-CN)": "zh",
    "Czech (cs)": "cs",
    "Danish (da)": "da",
    "Dutch (nl)": "nl",
    "English (en)": "en",
    "Finnish (fi)": "fi",
    "French (fr)": "fr",
    "German (de)": "de",
    "Greek (el)": "el",
    "Hebrew (he)": "he",
    "Hungarian (hu)": "hu",
    "Italian (it)": "it",
    "Japanese (ja)": "ja",
    "Korean (ko)": "ko",
    "Persian (fa)": "fa",  # no aux gTTS
    "Polish (pl)": "pl",
    "Portuguese (pt)": "pt",
    "Russian (ru)": "ru",
    "Spanish (es)": "es",
    "Turkish (tr)": "tr",
    "Ukrainian (uk)": "uk",
    "Urdu (ur)": "ur",
    "Vietnamese (vi)": "vi",
    "Hindi (hi)": "hi",
    "Indonesian (id)": "id",
    "Bengali (bn)": "bn",
    "Telugu (te)": "te",
    "Marathi (mr)": "mr",
    "Tamil (ta)": "ta",
    "Javanese (jw|jv)": "jw",
    "Catalan (ca)": "ca",
    "Nepali (ne)": "ne",
    "Thai (th)": "th",
    "Swedish (sv)": "sv",
    "Amharic (am)": "am",
    "Welsh (cy)": "cy",  # no aux gTTS
    "Estonian (et)": "et",
    "Croatian (hr)": "hr",
    "Icelandic (is)": "is",
    "Georgian (ka)": "ka",  # no aux gTTS
    "Khmer (km)": "km",
    "Slovak (sk)": "sk",
    "Albanian (sq)": "sq",
    "Serbian (sr)": "sr",
    "Azerbaijani (az)": "az",  # no aux gTTS
    "Bulgarian (bg)": "bg",
    "Galician (gl)": "gl",  # no aux gTTS
    "Gujarati (gu)": "gu",
    "Kazakh (kk)": "kk",  # no aux gTTS
    "Kannada (kn)": "kn",
    "Lithuanian (lt)": "lt",  # no aux gTTS
    "Latvian (lv)": "lv",
    "Macedonian (mk)": "mk",  # no aux gTTS # error get align model
    "Malayalam (ml)": "ml",
    "Malay (ms)": "ms",  # error get align model
    "Romanian (ro)": "ro",
    "Sinhala (si)": "si",
    "Sundanese (su)": "su",
    "Swahili (sw)": "sw",  # error aling
    "Afrikaans (af)": "af",
    "Bosnian (bs)": "bs",
    "Latin (la)": "la",
    "Myanmar Burmese (my)": "my",
    "Norwegian (no|nb)": "no",
    "Chinese - Traditional (zh-TW)": "zh-TW",
    "Assamese (as)": "as",
    "Basque (eu)": "eu",
    "Hausa (ha)": "ha",
    "Haitian Creole (ht)": "ht",
    "Armenian (hy)": "hy",
    "Lao (lo)": "lo",
    "Malagasy (mg)": "mg",
    "Mongolian (mn)": "mn",
    "Maltese (mt)": "mt",
    "Punjabi (pa)": "pa",
    "Pashto (ps)": "ps",
    "Slovenian (sl)": "sl",
    "Shona (sn)": "sn",
    "Somali (so)": "so",
    "Tajik (tg)": "tg",
    "Turkmen (tk)": "tk",
    "Tatar (tt)": "tt",
    "Uzbek (uz)": "uz",
    "Yoruba (yo)": "yo",
    **LANGUAGES_UNIDIRECTIONAL
}
BASE_L_LIST = LANGUAGES.keys()
LANGUAGES_LIST = [list(BASE_L_LIST)[0]] + sorted(list(BASE_L_LIST)[1:])
EXTRA_ALIGN = {
    "id": "indonesian-nlp/wav2vec2-large-xlsr-indonesian",
    "bn": "arijitx/wav2vec2-large-xlsr-bengali",
    "mr": "sumedh/wav2vec2-large-xlsr-marathi",
    "ta": "Amrrs/wav2vec2-large-xlsr-53-tamil",
    "jw": "cahya/wav2vec2-large-xlsr-javanese",
    "ne": "shniranjan/wav2vec2-large-xlsr-300m-nepali",
    "th": "sakares/wav2vec2-large-xlsr-thai-demo",
    "sv": "KBLab/wav2vec2-large-voxrex-swedish",
    "am": "agkphysics/wav2vec2-large-xlsr-53-amharic",
    "cy": "Srulikbdd/Wav2Vec2-large-xlsr-welsh",
    "et": "anton-l/wav2vec2-large-xlsr-53-estonian",
    "hr": "classla/wav2vec2-xls-r-parlaspeech-hr",
    "is": "carlosdanielhernandezmena/wav2vec2-large-xlsr-53-icelandic-ep10-1000h",
    "ka": "MehdiHosseiniMoghadam/wav2vec2-large-xlsr-53-Georgian",
    "km": "vitouphy/wav2vec2-xls-r-300m-khmer",
    "sk": "infinitejoy/wav2vec2-large-xls-r-300m-slovak",
    "sq": "Alimzhan/wav2vec2-large-xls-r-300m-albanian-colab",
    "sr": "dnikolic/wav2vec2-xlsr-530-serbian-colab",
    "az": "nijatzeynalov/wav2vec2-large-mms-1b-azerbaijani-common_voice15.0",
    "bg": "infinitejoy/wav2vec2-large-xls-r-300m-bulgarian",
    "gl": "ifrz/wav2vec2-large-xlsr-galician",
    "gu": "Harveenchadha/vakyansh-wav2vec2-gujarati-gnm-100",
    "kk": "aismlv/wav2vec2-large-xlsr-kazakh",
    "kn": "Harveenchadha/vakyansh-wav2vec2-kannada-knm-560",
    "lt": "DeividasM/wav2vec2-large-xlsr-53-lithuanian",
    "lv": "anton-l/wav2vec2-large-xlsr-53-latvian",
    "mk": "",  # Konstantin-Bogdanoski/wav2vec2-macedonian-base
    "ml": "gvs/wav2vec2-large-xlsr-malayalam",
    "ms": "",  # Duy/wav2vec2_malay
    "ro": "anton-l/wav2vec2-large-xlsr-53-romanian",
    "si": "IAmNotAnanth/wav2vec2-large-xls-r-300m-sinhala",
    "su": "cahya/wav2vec2-large-xlsr-sundanese",
    "sw": "",  # Lians/fine-tune-wav2vec2-large-swahili
    "af": "",  # ylacombe/wav2vec2-common_voice-af-demo
    "bs": "",
    "la": "",
    "my": "",
    "no": "NbAiLab/wav2vec2-xlsr-300m-norwegian",
    "zh-TW": "jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn",
    "as": "",
    "eu": "", # cahya/wav2vec2-large-xlsr-basque # verify
    "ha": "infinitejoy/wav2vec2-large-xls-r-300m-hausa",
    "ht": "",
    "hy": "infinitejoy/wav2vec2-large-xls-r-300m-armenian", # no (.)
    "lo": "",
    "mg": "",
    "mn": "tugstugi/wav2vec2-large-xlsr-53-mongolian",
    "mt": "carlosdanielhernandezmena/wav2vec2-large-xlsr-53-maltese-64h",
    "pa": "kingabzpro/wav2vec2-large-xlsr-53-punjabi",
    "ps": "aamirhs/wav2vec2-large-xls-r-300m-pashto-colab",
    "sl": "anton-l/wav2vec2-large-xlsr-53-slovenian",
    "sn": "",
    "so": "",
    "tg": "",
    "tk": "",  # Ragav/wav2vec2-tk
    "tt": "anton-l/wav2vec2-large-xlsr-53-tatar",
    "uz": "",  # Mekhriddin/wav2vec2-large-xls-r-300m-uzbek-colab
    "yo": "ogbi/wav2vec2-large-mms-1b-yoruba-test",
}


INVERTED_LANGUAGES = {value: key for key, value in LANGUAGES.items()}
TRANSLATION_PROCESS_OPTIONS = [
    "google_translator_batch",
    "google_translator",
]

def transcribe_speech(audio_path,language_name=None):
  asr_options = {
        "initial_prompt": None,
        "suppress_numerals": True
    }

  whisperx_model = whisperx.load_model(
        "large-v3",
        "cuda" ,
        compute_type="auto",
        language=language_name,
        asr_options=asr_options,
    )
  audio = whisperx.load_audio(audio_path)
  result = whisperx_model.transcribe(
      audio,
      batch_size=16,
      chunk_size=15,
      print_progress=True,
  )
  del whisperx_model
  gc.collect()
  torch.cuda.empty_cache()  # noqa
  return audio, result

def align_speech(audio, result):
    """
    Aligns speech segments based on the provided audio and result metadata.

    Parameters:
    - audio (array): The audio data in a suitable format for alignment.
    - result (dict): Metadata containing information about the segments
         and language.

    Returns:
    - result (dict): Updated metadata after aligning the segments with
        the audio. This includes character-level alignments if
        'return_char_alignments' is set to True.

    Notes:
    - This function uses language-specific models to align speech segments.
    - It performs language compatibility checks and selects the
        appropriate alignment model.
    - Cleans up memory by releasing resources after alignment.
    """
    global LANGUAGES,EXTRA_ALIGN,INVERTED_LANGUAGES
    my_lang= result["language"]
    DAMHF.update(DAMT)  # lang align
    if (
        not result["language"] in DAMHF.keys()
        and not result["language"] in EXTRA_ALIGN.keys()
    ):
        print(
            "Automatic detection: Source language not compatible with align"
        )
        # raise ValueError(
        #     f"Detected language {result['language']}  incompatible, "
        #     "you can select the source language to avoid this error."
        # )
    if (
        result["language"] in EXTRA_ALIGN.keys()
        and EXTRA_ALIGN[result["language"]] == ""
    ):
        lang_name = (
            INVERTED_LANGUAGES[result["language"]]
            if result["language"] in INVERTED_LANGUAGES.keys()
            else result["language"]
        )
        # logger.warning(
        #     "No compatible wav2vec2 model found "
        #     f"for the language '{lang_name}', skipping alignment."
        # )
        return result

    model_a, metadata = whisperx.load_align_model(
        language_code=result["language"],
        device="cuda",
        model_name=None
        if result["language"] in DAMHF.keys()
        else EXTRA_ALIGN[result["language"]],
    )
    result = whisperx.align(
        result["segments"],
        model_a,
        metadata,
        audio,
        "cuda",
        return_char_alignments=True,
        print_progress=False,
    )
    del model_a
    gc.collect()
    torch.cuda.empty_cache()  # noqa
    return result,my_lang


def linguistic_level_segments(
    result_base,
    linguistic_unit="word",  # word or char
):
    linguistic_unit = linguistic_unit[:4]
    linguistic_unit_key = linguistic_unit + "s"
    result = copy.deepcopy(result_base)

    if linguistic_unit_key not in result["segments"][0].keys():
        raise ValueError("No alignment detected, can't process")

    segments_by_unit = []
    for segment in result["segments"]:
        segment_units = segment[linguistic_unit_key]
        # segment_speaker = segment.get("speaker", "SPEAKER_00")

        for unit in segment_units:

            text = unit[linguistic_unit]

            if "start" in unit.keys():
                segments_by_unit.append(
                    {
                        "start": unit["start"],
                        "end": unit["end"],
                        "text": text,
                        # "speaker": segment_speaker,
                    }
                    )
            elif not segments_by_unit:
                pass
            else:
                segments_by_unit[-1]["text"] += text

    return {"segments": segments_by_unit}

from whisperx.utils import get_writer
import os
import copy
import os

def extract_folder_and_filename(file_path):
    folder_path = os.path.dirname(file_path)
    filename_with_ext = os.path.basename(file_path)
    filename_without_ext = os.path.splitext(filename_with_ext)[0]
    return folder_path, filename_without_ext+"."


def process_subtitles(result,srt_full_path=""):
  subs_copy_result = copy.deepcopy(result)
  for segment in subs_copy_result["segments"]:
      segment.pop("speaker", None)
  save_folder, name_ori = extract_folder_and_filename(srt_full_path)
  writer = get_writer("srt", output_dir=save_folder)
  word_options = {
      "highlight_words": False,
      "max_line_count": None,
      "max_line_width": None,
  }
  try:
    writer(
      subs_copy_result,
      name_ori[:-1] + ".mp3",
      word_options,
      )
  except:
    subs_copy_result["segments"][0].pop("words")
    writer(
              subs_copy_result,
              name_ori[:-1] + ".mp3",
              word_options,
          )
  return f"{save_folder}/{name_ori}srt"


def whisper_subtitle_generator(audio_path,Source_Language,Destination_Language):
  _,temp_name=extract_folder_and_filename(audio_path)
  temp_name=temp_name[:-1]
  if not os.path.exists("/content/subtitle/"):
    os.makedirs("/content/subtitle/")
  original_srt=f"/content/subtitle/Original_{Source_Language}_{temp_name}.srt"
  original_word_level_srt=f"/content/subtitle/Original_{Source_Language}_world_level_{temp_name}.srt"
  translated_srt=f"/content/subtitle/Translated_{Destination_Language}_{temp_name}.srt"
  if Source_Language=="None":
    audio, result=transcribe_speech(audio_path,None)
  else:
    audio, result=transcribe_speech(audio_path,Source_Language)
  result,get_lang=align_speech(audio, result)
  original_result = copy.deepcopy(result)
  process_subtitles(original_result,original_srt)
  wordl_level_result=linguistic_level_segments(result,linguistic_unit="word")
  process_subtitles(wordl_level_result,original_word_level_srt)
  Source_Language=get_lang
  if Source_Language==Destination_Language:
    return original_srt,original_word_level_srt,original_srt
  else:
    original_result = copy.deepcopy(result)
    translate_res=translate_whisper_text(result,Source_Language,Destination_Language)
    process_subtitles(translate_res,translated_srt)
    return original_srt,original_word_level_srt,translated_srt


import shutil
import re
import uuid
def clean_file_name(file_path):
  # Get the base file name and extension
  file_name = os.path.basename(file_path)
  file_name, file_extension = os.path.splitext(file_name)
  cleaned = re.sub(r'[^a-zA-Z\d]+', '_', file_name)   # Replace non-alphabetic characters with single underscore
  clean_file_name=re.sub(r'([a-zA-Z])_+([a-zA-Z])', r'\1_\2', cleaned)  # Replace multiple underscores between alphabets with one underscore
  clean_file_name = clean_file_name.strip('_')
  random_uuid = uuid.uuid4().hex[:6]
  # Combine cleaned file name with the original extension
  clean_file_path = os.path.join(os.path.dirname(file_path), clean_file_name +f"_{random_uuid}"+file_extension)
  return clean_file_path
def gradio_whisper_subtitle(gradio_file_path,Source_Language,Destination_Language):
  global languages
  audio_file_path=os.path.basename(gradio_file_path)
  audio_file_path=f"/content/gradio_upload/{audio_file_path}"
  audio_file_path=clean_file_name(audio_file_path)
  if not os.path.exists("/content/gradio_upload"):
    os.mkdir("/content/gradio_upload")
  shutil.copy(gradio_file_path,audio_file_path)

  if audio_file_path.lower().endswith(".mp4"):
    file_name = os.path.splitext(os.path.basename(audio_file_path))[0]
    temp_path=f"/content/gradio_upload/{file_name}.mp3"
    command=f"ffmpeg -i {audio_file_path} {temp_path}"
    var=os.system(command)
    if var==0:
      audio_path=temp_path
    else:
      print("Failed")
      print(command)
  else:
    audio_path=audio_file_path
  original_srt,original_word_level_srt,translated_srt=whisper_subtitle_generator(audio_path,languages[Source_Language],languages[Destination_Language])
  return original_srt,original_word_level_srt,translated_srt,

from IPython.display import clear_output
clear_output()

In [ ]:
#@title translate
from tqdm import tqdm
from itertools import chain
import copy
from deep_translator import GoogleTranslator

def translate_iterative(segments, target, source=None):
    """
    Translate text segments individually to the specified language.

    Parameters:
    - segments (list): A list of dictionaries with 'text' as a key for
        segment text.
    - target (str): Target language code.
    - source (str, optional): Source language code. Defaults to None.

    Returns:
    - list: Translated text segments in the target language.

    Notes:
    - Translates each segment using Google Translate.

    Example:
    segments = [{'text': 'first segment.'}, {'text': 'second segment.'}]
    translated_segments = translate_iterative(segments, 'es')
    """

    segments_ = copy.deepcopy(segments)

    if (
        not source
    ):
        print("No source language")
        source = "auto"

    translator = GoogleTranslator(source=source, target=target)
    for line in tqdm(range(len(segments_))):
        text = segments_[line]["text"]
        translated_line = translator.translate(text.strip())
        segments_[line]["text"] = translated_line

    return segments_


def verify_translate(
    segments,
    segments_copy,
    translated_lines,
    target,
    source
):
    """
    Verify integrity and translate segments if lengths match, otherwise
    switch to iterative translation.
    """
    if len(segments) == len(translated_lines):
        for line in range(len(segments_copy)):
            print(
                f"{segments_copy[line]['text']} >> "
                f"{translated_lines[line].strip()}"
            )
            segments_copy[line]["text"] = translated_lines[
                line].replace("\t", "").replace("\n", "").strip()
        return segments_copy
    else:
        print(
            "The translation failed, switching to google_translate iterative. "
            f"{len(segments), len(translated_lines)}"
        )
        return translate_iterative(segments, target, source)


def translate_batch(segments, target, chunk_size=2000, source=None):
    """
    Translate a batch of text segments into the specified language in chunks,
        respecting the character limit.

    Parameters:
    - segments (list): List of dictionaries with 'text' as a key for segment
        text.
    - target (str): Target language code.
    - chunk_size (int, optional): Maximum character limit for each translation
        chunk (default is 2000; max 5000).
    - source (str, optional): Source language code. Defaults to None.

    Returns:
    - list: Translated text segments in the target language.

    Notes:
    - Splits input segments into chunks respecting the character limit for
        translation.
    - Translates the chunks using Google Translate.
    - If chunked translation fails, switches to iterative translation using
        `translate_iterative()`.

    Example:
    segments = [{'text': 'first segment.'}, {'text': 'second segment.'}]
    translated = translate_batch(segments, 'es', chunk_size=4000, source='en')
    """

    segments_copy = copy.deepcopy(segments)

    if (
        not source
    ):
        print("No source language")
        source = "auto"

    # Get text
    text_lines = []
    for line in range(len(segments_copy)):
        text = segments_copy[line]["text"].strip()
        text_lines.append(text)

    # chunk limit
    text_merge = []
    actual_chunk = ""
    global_text_list = []
    actual_text_list = []
    for one_line in text_lines:
        one_line = " " if not one_line else one_line
        if (len(actual_chunk) + len(one_line)) <= chunk_size:
            if actual_chunk:
                actual_chunk += " ||||| "
            actual_chunk += one_line
            actual_text_list.append(one_line)
        else:
            text_merge.append(actual_chunk)
            actual_chunk = one_line
            global_text_list.append(actual_text_list)
            actual_text_list = [one_line]
    if actual_chunk:
        text_merge.append(actual_chunk)
        global_text_list.append(actual_text_list)

    # translate chunks
    progress_bar = tqdm(total=len(segments), desc="Translating")
    translator = GoogleTranslator(source=source, target=target)
    split_list = []
    try:
        for text, text_iterable in zip(text_merge, global_text_list):
            translated_line = translator.translate(text.strip())
            split_text = translated_line.split("|||||")
            if len(split_text) == len(text_iterable):
                progress_bar.update(len(split_text))
            else:
                print(
                    "Chunk fixing iteratively. Len chunk: "
                    f"{len(split_text)}, expected: {len(text_iterable)}"
                )
                split_text = []
                for txt_iter in text_iterable:
                    translated_txt = translator.translate(txt_iter.strip())
                    split_text.append(translated_txt)
                    progress_bar.update(1)
            split_list.append(split_text)
        progress_bar.close()
    except Exception as error:
        progress_bar.close()
        print(str(error))
        print(
            "The translation in chunks failed, switching to iterative."
            " Related: too many request"
        )  # use proxy or less chunk size
        return translate_iterative(segments, target, source)

    # un chunk
    translated_lines = list(chain.from_iterable(split_list))

    return verify_translate(
        segments, segments_copy, translated_lines, target, source
    )
def fix_code_language(translate_to, syntax="google"):
    if syntax == "google":
        # google-translator, gTTS
        replace_lang_code = {"zh": "zh-CN", "he": "iw", "zh-cn": "zh-CN"}
    elif syntax == "coqui":
        # coqui-xtts
        replace_lang_code = {"zh": "zh-cn", "zh-CN": "zh-cn", "zh-TW": "zh-cn"}

    new_code_lang = replace_lang_code.get(translate_to, translate_to)
    print(f"Fix code {translate_to} -> {new_code_lang}")
    return new_code_lang

def translate_text(
    segments,
    target,
    translation_process="google_translator_batch",
    chunk_size=4500,
    source=None,
    token_batch_limit=1000,
):
    """Translates text segments using a specified process."""
    match translation_process:
        case "google_translator_batch":
            return translate_batch(
                segments,
                fix_code_language(target),
                chunk_size,
                fix_code_language(source)
            )
        case "google_translator":
            return translate_iterative(
                segments,
                fix_code_language(target),
                fix_code_language(source)
            )

def translate_whisper_text(result,source_lang,destination_lang):
  copy_result = copy.deepcopy(result)
  translate_process="google_translator"
  lang_source=source_lang
  TRANSLATE_AUDIO_TO=destination_lang
  copy_result['segments']=translate_text(copy_result["segments"],TRANSLATE_AUDIO_TO,translate_process,chunk_size=1800)
  return copy_result

In [ ]:
#@title Edge tts Config and demo
from deep_translator import GoogleTranslator

def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"
languages = {
    "None": "None",
    "Afrikaans": "af",
    "Amharic": "am",
    "Arabic": "ar",
    "Azerbaijani": "az",
    "Bulgarian": "bg",
    "Bengali": "bn",
    "Bosnian": "bs",
    "Catalan": "ca",
    "Czech": "cs",
    "Welsh": "cy",
    "Danish": "da",
    "German": "de",
    "Greek": "el",
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "Irish": "ga",
    "Galician": "gl",
    "Gujarati": "gu",
    "Hebrew": "he",
    "Hindi": "hi",
    "Croatian": "hr",
    "Hungarian": "hu",
    "Indonesian": "id",
    "Icelandic": "is",
    "Italian": "it",
    "Japanese": "ja",
    "Javanese": "jv",
    "Georgian": "ka",
    "Kazakh": "kk",
    "Khmer": "km",
    "Kannada": "kn",
    "Korean": "ko",
    "Lao": "lo",
    "Lithuanian": "lt",
    "Latvian": "lv",
    "Macedonian": "mk",
    "Malayalam": "ml",
    "Mongolian": "mn",
    "Marathi": "mr",
    "Malay": "ms",
    "Maltese": "mt",
    "Burmese": "my",
    "Norwegian Bokmål": "nb",
    "Nepali": "ne",
    "Dutch": "nl",
    "Polish": "pl",
    "Pashto": "ps",
    "Portuguese": "pt",
    "Romanian": "ro",
    "Russian": "ru",
    "Sinhala": "si",
    "Slovak": "sk",
    "Slovenian": "sl",
    "Somali": "so",
    "Albanian": "sq",
    "Serbian": "sr",
    "Sundanese": "su",
    "Swedish": "sv",
    "Swahili": "sw",
    "Tamil": "ta",
    "Telugu": "te",
    "Thai": "th",
    "Turkish": "tr",
    "Ukrainian": "uk",
    "Urdu": "ur",
    "Uzbek": "uz",
    "Vietnamese": "vi",
    "Chinese": "zh",
    "Zulu": "zu"
}





def translate_text(text, Language):
    target_language=languages[Language]
    # if Language == "English" :
    #   target_language='en'
    # if Language == "Hindi":
    #   target_language='hi'
    # if Language == "Bengali":
    #   target_language='bn'
    if Language == "Chinese":
          target_language='zh-CN'
    translator = GoogleTranslator(target=target_language)
    translation = translator.translate(text.strip())
    t_text=str(translation)
    if Language == "English" :
      return t_text
    elif Language == "Hindi" or Language == "Bengali":
      return t_text.replace(".","।")
    else:
      return t_text


def make_chunks(input_text, language):
    return [input_text]
    # if language == "English":
    #     temp_list = input_text.strip().split(".")
    #     filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
    #     if temp_list[-1].strip():
    #         filtered_list.append(temp_list[-1].strip())
    #     return filtered_list

    # elif language == "Hindi" or language == "Bengali":
    #     temp_list = input_text.strip().split("।")
    #     filtered_list = [element.strip() + '।' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
    #     if temp_list[-1].strip():
    #         filtered_list.append(temp_list[-1].strip())
    #     return filtered_list
    # else:
    #   return [input_text]



import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # voice_name="en-IE-EmilyNeural"  # @param {type: "string"}
#   print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      # print(i)
      edge_command=f'''edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{text}" --write-media {save_path}'''
      # edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'

      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
        print(edge_command)
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    # print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path
female_voice_list={'Vietnamese': 'vi-VN-HoaiMyNeural',
 'Bengali': 'bn-BD-NabanitaNeural',
 'Thai': 'th-TH-PremwadeeNeural',
 'English': 'en-AU-NatashaNeural',
 'Portuguese': 'pt-BR-FranciscaNeural',
 'Arabic': 'ar-AE-FatimaNeural',
 'Turkish': 'tr-TR-EmelNeural',
 'Spanish': 'es-AR-ElenaNeural',
 'Korean': 'ko-KR-SunHiNeural',
 'French': 'fr-BE-CharlineNeural',
 'Indonesian': 'id-ID-GadisNeural',
 'Russian': 'ru-RU-SvetlanaNeural',
 'Hindi': 'hi-IN-SwaraNeural',
 'Japanese': 'ja-JP-NanamiNeural',
 'Afrikaans': 'af-ZA-AdriNeural',
 'Amharic': 'am-ET-MekdesNeural',
 'Azerbaijani': 'az-AZ-BanuNeural',
 'Bulgarian': 'bg-BG-KalinaNeural',
 'Bosnian': 'bs-BA-VesnaNeural',
 'Catalan': 'ca-ES-JoanaNeural',
 'Czech': 'cs-CZ-VlastaNeural',
 'Welsh': 'cy-GB-NiaNeural',
 'Danish': 'da-DK-ChristelNeural',
 'German': 'de-AT-IngridNeural',
 'Greek': 'el-GR-AthinaNeural',
 'Irish': 'ga-IE-OrlaNeural',
 'Galician': 'gl-ES-SabelaNeural',
 'Gujarati': 'gu-IN-DhwaniNeural',
 'Hebrew': 'he-IL-HilaNeural',
 'Croatian': 'hr-HR-GabrijelaNeural',
 'Hungarian': 'hu-HU-NoemiNeural',
 'Icelandic': 'is-IS-GudrunNeural',
 'Italian': 'it-IT-ElsaNeural',
 'Javanese': 'jv-ID-SitiNeural',
 'Georgian': 'ka-GE-EkaNeural',
 'Kazakh': 'kk-KZ-AigulNeural',
 'Khmer': 'km-KH-SreymomNeural',
 'Kannada': 'kn-IN-SapnaNeural',
 'Lao': 'lo-LA-KeomanyNeural',
 'Lithuanian': 'lt-LT-OnaNeural',
 'Latvian': 'lv-LV-EveritaNeural',
 'Macedonian': 'mk-MK-MarijaNeural',
 'Malayalam': 'ml-IN-SobhanaNeural',
 'Mongolian': 'mn-MN-YesuiNeural',
 'Marathi': 'mr-IN-AarohiNeural',
 'Malay': 'ms-MY-YasminNeural',
 'Maltese': 'mt-MT-GraceNeural',
 'Burmese': 'my-MM-NilarNeural',
 'Norwegian Bokmål': 'nb-NO-PernilleNeural',
 'Nepali': 'ne-NP-HemkalaNeural',
 'Dutch': 'nl-BE-DenaNeural',
 'Polish': 'pl-PL-ZofiaNeural',
 'Pashto': 'ps-AF-LatifaNeural',
 'Romanian': 'ro-RO-AlinaNeural',
 'Sinhala': 'si-LK-ThiliniNeural',
 'Slovak': 'sk-SK-ViktoriaNeural',
 'Slovenian': 'sl-SI-PetraNeural',
 'Somali': 'so-SO-UbaxNeural',
 'Albanian': 'sq-AL-AnilaNeural',
 'Serbian': 'sr-RS-SophieNeural',
 'Sundanese': 'su-ID-TutiNeural',
 'Swedish': 'sv-SE-SofieNeural',
 'Swahili': 'sw-KE-ZuriNeural',
 'Tamil': 'ta-IN-PallaviNeural',
 'Telugu': 'te-IN-ShrutiNeural',
 'Chinese': 'zh-CN-XiaoxiaoNeural',
 'Ukrainian': 'uk-UA-PolinaNeural',
 'Urdu': 'ur-IN-GulNeural',
 'Uzbek': 'uz-UZ-MadinaNeural',
 'Zulu': 'zu-ZA-ThandoNeural'}
male_voice_list= {'Vietnamese': 'vi-VN-NamMinhNeural',
 'Bengali': 'bn-BD-PradeepNeural',
 'Thai': 'th-TH-NiwatNeural',
 'English': 'en-AU-WilliamNeural',
 'Portuguese': 'pt-BR-AntonioNeural',
 'Arabic': 'ar-AE-HamdanNeural',
 'Turkish': 'tr-TR-AhmetNeural',
 'Spanish': 'es-AR-TomasNeural',
 'Korean': 'ko-KR-HyunsuNeural',
 'French': 'fr-BE-GerardNeural',
 'Indonesian': 'id-ID-ArdiNeural',
 'Russian': 'ru-RU-DmitryNeural',
 'Hindi': 'hi-IN-MadhurNeural',
 'Japanese': 'ja-JP-KeitaNeural',
 'Afrikaans': 'af-ZA-WillemNeural',
 'Amharic': 'am-ET-AmehaNeural',
 'Azerbaijani': 'az-AZ-BabekNeural',
 'Bulgarian': 'bg-BG-BorislavNeural',
 'Bosnian': 'bs-BA-GoranNeural',
 'Catalan': 'ca-ES-EnricNeural',
 'Czech': 'cs-CZ-AntoninNeural',
 'Welsh': 'cy-GB-AledNeural',
 'Danish': 'da-DK-JeppeNeural',
 'German': 'de-AT-JonasNeural',
 'Greek': 'el-GR-NestorasNeural',
 'Irish': 'ga-IE-ColmNeural',
 'Galician': 'gl-ES-RoiNeural',
 'Gujarati': 'gu-IN-NiranjanNeural',
 'Hebrew': 'he-IL-AvriNeural',
 'Croatian': 'hr-HR-SreckoNeural',
 'Hungarian': 'hu-HU-TamasNeural',
 'Icelandic': 'is-IS-GunnarNeural',
 'Italian': 'it-IT-DiegoNeural',
 'Javanese': 'jv-ID-DimasNeural',
 'Georgian': 'ka-GE-GiorgiNeural',
 'Kazakh': 'kk-KZ-DauletNeural',
 'Khmer': 'km-KH-PisethNeural',
 'Kannada': 'kn-IN-GaganNeural',
 'Lao': 'lo-LA-ChanthavongNeural',
 'Lithuanian': 'lt-LT-LeonasNeural',
 'Latvian': 'lv-LV-NilsNeural',
 'Macedonian': 'mk-MK-AleksandarNeural',
 'Malayalam': 'ml-IN-MidhunNeural',
 'Mongolian': 'mn-MN-BataaNeural',
 'Marathi': 'mr-IN-ManoharNeural',
 'Malay': 'ms-MY-OsmanNeural',
 'Maltese': 'mt-MT-JosephNeural',
 'Burmese': 'my-MM-ThihaNeural',
 'Norwegian Bokmål': 'nb-NO-FinnNeural',
 'Nepali': 'ne-NP-SagarNeural',
 'Dutch': 'nl-BE-ArnaudNeural',
 'Polish': 'pl-PL-MarekNeural',
 'Pashto': 'ps-AF-GulNawazNeural',
 'Romanian': 'ro-RO-EmilNeural',
 'Sinhala': 'si-LK-SameeraNeural',
 'Slovak': 'sk-SK-LukasNeural',
 'Slovenian': 'sl-SI-RokNeural',
 'Somali': 'so-SO-MuuseNeural',
 'Albanian': 'sq-AL-IlirNeural',
 'Serbian': 'sr-RS-NicholasNeural',
 'Sundanese': 'su-ID-JajangNeural',
 'Swedish': 'sv-SE-MattiasNeural',
 'Swahili': 'sw-KE-RafikiNeural',
 'Tamil': 'ta-IN-ValluvarNeural',
 'Telugu': 'te-IN-MohanNeural',
 'Chinese': 'zh-CN-YunjianNeural',
 'Ukrainian': 'uk-UA-OstapNeural',
 'Urdu': 'ur-IN-SalmanNeural',
 'Uzbek': 'uz-UZ-SardorNeural',
 'Zulu': 'zu-ZA-ThembaNeural'}
text = 'Hi, How are you .'  # @param {type: "string"}
Language = "Spanish" # @param ['Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']
Gender = "Male"# @param ['Male', 'Female']
speed = 1  # @param {type: "number"}

translate_text_flag  = True # @param {type:"boolean"}
# long_sentence = True # @param {type:"boolean"}
long_sentence=False
save_path = '/content/edge.wav'  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    # voice_name="en-US-ChristopherNeural"
    voice_name="en-US-BrianNeural"
  if Gender=="Female":
    voice_name="en-US-AriaNeural"
elif Language == "Hindi":
  if Gender=="Male":
    voice_name="hi-IN-MadhurNeural"
  if Gender=="Female":
    voice_name="hi-IN-SwaraNeural"
elif Language == "Bengali":
  if Gender=="Male":
    voice_name="bn-IN-BashkarNeural"
  if Gender=="Female":
    voice_name="bn-BD-NabanitaNeural"
else:
  if Gender=="Male":
    voice_name=male_voice_list[Language]
  if Gender=="Female":
    voice_name=female_voice_list[Language]
if translate_text_flag:
  input_text=translate_text(text, Language)
#   print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# print(chunks_list,speed,voice_name,save_path)
edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)





# remove_slience  = True # @param {type:"boolean"}
# slience_margin = 0.1  # @param {type: "number"}
remove_slience  = True
if remove_slience:
  new_file_path=edge_save_path
  # new_file_path,_=remove_silence_from_audio(edge_save_path, slience_margin)
else:
  new_file_path=edge_save_path
auto_download  = False # @param {type:"boolean"}
from google.colab import files
if auto_download:
  files.download(new_file_path)
from IPython.display import clear_output
clear_output()


def process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag):
  if Gender=="Male":
    voice_name=male_voice_list[Language]
  if Gender=="Female":
    voice_name=female_voice_list[Language]
  if translate_text_flag:
    input_text=translate_text(text, Language)
    # print("Translateting")
  else:
    input_text=text
  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  edge_free_tts(chunks_list,speed,voice_name,audio_path)
# text="hi how are you"
# speed=1
# audio_path='/content/test.mp3'
# Language='English'
# Gender='Male'
# long_sentence=True
# translate_text_flag=True
# process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag)
# Audio(audio_path, autoplay=True)





from IPython.display import Audio
Audio(new_file_path, autoplay=True)

In [ ]:
# @title ##**Choose option [select video]** { display-mode: "form" }
choose= "already_uploaded"#@param ['upload_now','already_uploaded']
import os

folder_path = "/content/gdrive/MyDrive/upload"

# Check if the folder exists
if not os.path.exists(folder_path):
    # If it doesn't exist, create it
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")

%cd /content/gdrive/MyDrive/upload
from IPython.display import clear_output
clear_output()
f_names=[]
from google.colab import files
if choose== "upload_now":
  uploaded = files.upload()
  for fn in uploaded.keys():
        f_names.append(fn)
  clear_output()

video_id=1
table={}
ids=[]
videos=[]
id_monitor={}
video_folder="/content/gdrive/MyDrive/upload"
for i in os.listdir(video_folder):
  ids.append(video_id)
  videos.append(i)
  id_monitor[video_id]=i
  video_id=video_id+1
table["file_name"]=videos
table["file_id"]=ids
import pandas as pd
df = pd.DataFrame(table)
try:
  df.set_index('file_id', inplace=True)
except:
  pass
print(df)
print("\n")


                                                 file_name
file_id                                                   
1        English_original_en_world_level_2024_07_19_18_...
2        English_original_en_2024_07_19_18_03_42_59bb69...
3                                                video.mp4




In [ ]:
# @title Enter input File ID
drive_file_upload_path=""
if choose== "already_uploaded":
  file_id =   3#@param {type: "number"}
  drive_file_upload_path="/content/gdrive/MyDrive/upload/"+id_monitor[int(file_id)]
drive_file_upload_path

'/content/gdrive/MyDrive/upload/video.mp4'

In [33]:
#@title Generate SRT
audio_file_path = '/content/gdrive/MyDrive/upload/video.mp4'  # @param {type: "string"}
Source_Language = "English" # @param ['None','English','Hindi','Bengali','Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']
Destination_Language = "Hindi" # @param ['English','Hindi','Bengali','Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']


original_srt,original_word_level_srt,translated_srt=gradio_whisper_subtitle(audio_file_path,Source_Language,Destination_Language)
from IPython.display import clear_output
clear_output()
from google.colab import files
files.download(original_srt)
files.download(original_word_level_srt)
files.download(translated_srt)
print(original_srt)
print(original_word_level_srt)
print(translated_srt)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/subtitle/Original_en_video_c8b734.srt
/content/subtitle/Original_en_world_level_video_c8b734.srt
/content/subtitle/Translated_hi_video_c8b734.srt


In [42]:
#@title Dub Subtitle in Different Languages

def process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag):
  if Gender=="Male":
    voice_name=male_voice_list[Language]
  if Gender=="Female":
    voice_name=female_voice_list[Language]
  if translate_text_flag:
    input_text=translate_text(text, Language)
    print("Translateting")
  else:
    input_text=text
  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  edge_free_tts(chunks_list,speed,voice_name,audio_path)

import os
def get_subtitle_Dub_path(srt_file_path,Language):
  file_name = os.path.splitext(os.path.basename(srt_file_path))[0]
  if not os.path.exists("/content/TTS_DUB"):
    os.mkdir("/content/TTS_DUB")
  new_path=f"/content/TTS_DUB/{Language}_{file_name}.wav"
  return new_path






import subprocess
import json

def get_video_duration(video_path):
    try:
        # Run ffmpeg command to get video information in JSON format
        result = subprocess.run(
            ['ffmpeg', '-i', video_path, '-f', 'ffmetadata', '-'],
            stderr=subprocess.PIPE,
            text=True
        )

        # Parse the duration from the stderr output
        for line in result.stderr.split('\n'):
            if 'Duration' in line:
                duration_str = line.split('Duration: ')[1].split(',')[0]
                h, m, s = duration_str.split(':')
                duration = int(h) * 3600 + int(m) * 60 + float(s)
                return duration
    except Exception as e:
        print(f"Error: {e}")
        return None


def replace_audio(video_path,audio_path):
  if not video_path.lower().endswith(".mp4"):
    return
  tts_audio = AudioSegment.from_file(dub_save_path)
  audio_duration = len(tts_audio)/1000
  video_duration=get_video_duration(video_path)
  slience_duration=video_duration-audio_duration
  audio_segment = AudioSegment.from_file(audio_path)
  slience_Segment= AudioSegment.silent(duration=slience_duration)
  marge_audio=audio_segment+slience_Segment
  marge_audio.export("/content/new_audio.wav", format="wav")
  command=f"ffmpeg -i {video_path}  -i /content/new_audio.wav -map 0:v -map 1:a -c:v copy -shortest /content/output.mp4 -y"
  var=os.system(command)
  if var==0:
    if os.path.exists("/content/gdrive/MyDrive/upload"):
      file_name = os.path.basename(video_path)
      shutil.copy("/content/output.mp4", f"/content/gdrive/MyDrive/upload/change_audio_{file_name}")
      print(f"Copied at drive '/content/gdrive/MyDrive/upload/change_audio_{file_name}'")
      return f"/content/gdrive/MyDrive/upload/change_audio_{file_name}"
  else:
    print(command)
    return None








###

from pydub import AudioSegment
import shutil
import subprocess
import os
import uuid
import re


srt_file_path = '/content/subtitle/Translated_hi_video_c8b734.srt'  # @param {type: "string"}
# dub_save_path = '/content/final.wav' # @param {type: "string"}
Language = "Hindi" # @param ['Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']
dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
import time
def your_tts(text,audio_path,language):
  Language=language
  Gender = "Male"# @param ['Male', 'Female']
  speed = 1  # @param {type: "number"}
  # long_sentence=True #@param {type:"boolean"}
  long_sentence=False
  translate_text_flag=False# @param {type:"boolean"}
  process_tts(text,speed,audio_path,Language,Gender,long_sentence,translate_text_flag)
  if long_sentence:
    time.sleep(1)

class SRTDubbing:
    def __init__(self):
        pass

    @staticmethod
    def text_to_speech(text, audio_path, language, actual_duration):
        tts_filename = "temp.wav"
        your_tts(text,tts_filename,language)
        # Check the duration of the generated TTS audio
        tts_audio = AudioSegment.from_file(tts_filename)
        tts_duration = len(tts_audio)

        if actual_duration == 0:
            # If actual duration is zero, use the original TTS audio without modifications
            shutil.move(tts_filename, audio_path)
            return

        # If TTS audio duration is longer than actual duration, speed up the audio
        if tts_duration > actual_duration:
            speedup_factor = tts_duration / actual_duration
            speedup_filename = "speedup_temp.wav"

            # Use ffmpeg to change audio speed
            subprocess.run([
                "ffmpeg",
                "-i", tts_filename,
                "-filter:a", f"atempo={speedup_factor}",
                speedup_filename
            ], check=True)

            # Replace the original TTS audio with the sped-up version
            shutil.move(speedup_filename, audio_path)
        elif tts_duration < actual_duration:
            # If TTS audio duration is less than actual duration, add silence to match the duration
            silence_gap = actual_duration - tts_duration
            silence = AudioSegment.silent(duration=int(silence_gap))
            new_audio = tts_audio + silence

            # Save the new audio with added silence
            new_audio.export(audio_path, format="wav")
        else:
            # If TTS audio duration is equal to actual duration, use the original TTS audio
            shutil.move(tts_filename, audio_path)

    @staticmethod
    def make_silence(pause_time, pause_save_path):
        silence = AudioSegment.silent(duration=pause_time)
        silence.export(pause_save_path, format="wav")
        return pause_save_path

    @staticmethod
    def create_folder_for_srt(srt_file_path):
        srt_base_name = os.path.splitext(os.path.basename(srt_file_path))[0]
        random_uuid = str(uuid.uuid4())[:4]
        dummy_folder_path = "/content/dummy"
        if not os.path.exists(dummy_folder_path):
            os.makedirs(dummy_folder_path)
        folder_path = os.path.join(dummy_folder_path, f"{srt_base_name}_{random_uuid}")
        os.makedirs(folder_path, exist_ok=True)
        return folder_path

    @staticmethod
    def concatenate_audio_files(audio_paths, output_path):
        concatenated_audio = AudioSegment.silent(duration=0)
        for audio_path in audio_paths:
            audio_segment = AudioSegment.from_file(audio_path)
            concatenated_audio += audio_segment
        concatenated_audio.export(output_path, format="wav")

    def srt_to_dub(self, srt_file_path, dub_save_path,language='en'):
        result = self.read_srt_file(srt_file_path)
        new_folder_path = self.create_folder_for_srt(srt_file_path)
        join_path = []
        for i in result:
            text = i['text']
            actual_duration = i['end_time'] - i['start_time']
            pause_time = i['pause_time']
            slient_path = f"{new_folder_path}/{i['previous_pause']}"
            self.make_silence(pause_time, slient_path)
            join_path.append(slient_path)
            tts_path = f"{new_folder_path}/{i['audio_name']}"
            self.text_to_speech(text, tts_path, language, actual_duration)
            join_path.append(tts_path)
        self.concatenate_audio_files(join_path, dub_save_path)

    @staticmethod
    def convert_to_millisecond(time_str):
      if isinstance(time_str, str):
          hours, minutes, second_millisecond = time_str.split(':')
          seconds, milliseconds = second_millisecond.split(",")

          total_milliseconds = (
              int(hours) * 3600000 +
              int(minutes) * 60000 +
              int(seconds) * 1000 +
              int(milliseconds)
          )

          return total_milliseconds
    @staticmethod
    def read_srt_file(file_path):
        entries = []
        default_start = 0
        previous_end_time = default_start
        entry_number = 1
        audio_name_template = "{}.wav"
        previous_pause_template = "{}_before_pause.wav"

        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            print(lines)
            for i in range(0, len(lines), 4):
                time_info = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', lines[i + 1])
                start_time = SRTDubbing.convert_to_millisecond(time_info[0][0])
                end_time = SRTDubbing.convert_to_millisecond(time_info[0][1])

                current_entry = {
                    'entry_number': entry_number,
                    'start_time': start_time,
                    'end_time': end_time,
                    'text': lines[i + 2].strip(),
                    'pause_time': start_time - previous_end_time if entry_number != 1 else start_time - default_start,
                    'audio_name': audio_name_template.format(entry_number),
                    'previous_pause': previous_pause_template.format(entry_number),
                }

                entries.append(current_entry)
                previous_end_time = end_time
                entry_number += 1

        return entries

# Example usage
srt_dubbing = SRTDubbing()
current_language=Language
srt_dubbing.srt_to_dub(srt_file_path, dub_save_path,current_language)
from IPython.display import clear_output
clear_output()
print(f"{Language} Dub Audio File Save At : {dub_save_path}")
if os.path.exists("/content/gdrive/MyDrive/upload"):
  file_name = os.path.basename(dub_save_path)
  shutil.copy(dub_save_path, f"/content/gdrive/MyDrive/upload/{file_name}")
  print(f"Copied at drive '/content/gdrive/MyDrive/upload/{file_name}'")


replace_audio_path=replace_audio(audio_file_path,dub_save_path)

from google.colab import files

files.download(dub_save_path)
files.download(replace_audio_path)



Hindi Dub Audio File Save At : /content/TTS_DUB/Hindi_Translated_hi_video_c8b734.wav
Copied at drive '/content/gdrive/MyDrive/upload/Hindi_Translated_hi_video_c8b734.wav'
Copied at drive '/content/gdrive/MyDrive/upload/video.mp4'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
#@title Dub subtitle new
# import os
# import re
# import shutil
# import subprocess
# import uuid
# from pydub import AudioSegment
# # from tqdm import tqdm
# from tqdm.notebook import tqdm as tqdm_colab

# import pysrt

# def clean_srt(input_path):
#     file_name = os.path.basename(input_path)
#     output_folder = "/content/save_srt"
#     if not os.path.exists(output_folder):
#         os.mkdir(output_folder)
#     output_path = f"{output_folder}/{file_name}"

#     def clean_srt_line(text):
#         bad_list = ["[", "]", "♫", "\n"]
#         for i in bad_list:
#             text = text.replace(i, "")
#         return text.strip()

#     # Load the subtitle file
#     subs = pysrt.open(input_path)

#     # Iterate through each subtitle and print its details
#     with open(output_path, "w", encoding='utf-8') as file:
#         for sub in subs:
#             file.write(f"{sub.index}\n")
#             file.write(f"{sub.start} --> {sub.end}\n")
#             file.write(f"{clean_srt_line(sub.text)}\n")
#             file.write("\n")
#         file.close()
#     # print(f"Clean SRT saved at: {output_path}")
#     return output_path
# # Example usage
# Enter_srt_file_path = '/content/subtitle/hi_video_74901d.srt'  # @param {type: "string"}
# srt_file_path=clean_srt(Enter_srt_file_path)
# Language = "Hindi" # @param ['English','Hindi','Bengali','Afrikaans', 'Amharic', 'Arabic', 'Azerbaijani', 'Bulgarian', 'Bengali', 'Bosnian', 'Catalan', 'Czech', 'Welsh', 'Danish', 'German', 'Greek', 'English', 'Spanish', 'French', 'Irish', 'Galician', 'Gujarati', 'Hebrew', 'Hindi', 'Croatian', 'Hungarian', 'Indonesian', 'Icelandic', 'Italian', 'Japanese', 'Javanese', 'Georgian', 'Kazakh', 'Khmer', 'Kannada', 'Korean', 'Lao', 'Lithuanian', 'Latvian', 'Macedonian', 'Malayalam', 'Mongolian', 'Marathi', 'Malay', 'Maltese', 'Burmese', 'Norwegian Bokmål', 'Nepali', 'Dutch', 'Polish', 'Pashto', 'Portuguese', 'Romanian', 'Russian', 'Sinhala', 'Slovak', 'Slovenian', 'Somali', 'Albanian', 'Serbian', 'Sundanese', 'Swedish', 'Swahili', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Chinese', 'Zulu']



# def process_tts(text, speed, audio_path, Language, Gender, long_sentence, translate_text_flag):
#     if Gender == "Male":
#         voice_name = male_voice_list[Language]
#     if Gender == "Female":
#         voice_name = female_voice_list[Language]
#     if translate_text_flag:
#         input_text = translate_text(text, Language)
#         # print("Translating")
#     else:
#         input_text = text
#     if long_sentence and translate_text_flag:
#         chunks_list = make_chunks(input_text, Language)
#     elif long_sentence:
#         chunks_list = make_chunks(input_text, "English")
#     else:
#         chunks_list = [input_text]
#     edge_free_tts(chunks_list, speed, voice_name, audio_path)


# def get_subtitle_Dub_path(srt_file_path, Language):
#     file_name = os.path.splitext(os.path.basename(srt_file_path))[0]
#     if not os.path.exists("/content/TTS_DUB"):
#         os.mkdir("/content/TTS_DUB")
#     new_path = f"/content/TTS_DUB/{Language}_{file_name}.wav"
#     return new_path


# def your_tts(text, audio_path, language):
#     Language = language
#     Gender = "Female"# @param ['Male', 'Female']
#     # speed = 1  # @param {type: "number"}
#     speed = 1
#     long_sentence = False
#     translate_text_flag=False# @param {type:"boolean"}
#     process_tts(text, speed, audio_path, Language, Gender, long_sentence, translate_text_flag)
#     if long_sentence:
#       time.sleep(1)

# class SRTDubbing:
#     def __init__(self):
#         pass

#     @staticmethod
#     def text_to_speech(text, audio_path, language, actual_duration):
#         tts_filename = "temp.wav"
#         your_tts(text, tts_filename, language)
#         return tts_filename

#     @staticmethod
#     def make_silence(pause_time, pause_save_path):
#         silence = AudioSegment.silent(duration=pause_time)
#         silence.export(pause_save_path, format="wav")
#         return pause_save_path

#     @staticmethod
#     def create_folder_for_srt(srt_file_path):
#         srt_base_name = os.path.splitext(os.path.basename(srt_file_path))[0]
#         random_uuid = str(uuid.uuid4())[:4]
#         dummy_folder_path = "/content/dummy"
#         if not os.path.exists(dummy_folder_path):
#             os.makedirs(dummy_folder_path)
#         folder_path = os.path.join(dummy_folder_path, f"{srt_base_name}_{random_uuid}")
#         os.makedirs(folder_path, exist_ok=True)
#         return folder_path

#     @staticmethod
#     def concatenate_audio_files(audio_paths, output_path):
#         concatenated_audio = AudioSegment.silent(duration=0)
#         for audio_path in audio_paths:
#             audio_segment = AudioSegment.from_file(audio_path)
#             concatenated_audio += audio_segment
#         concatenated_audio.export(output_path, format="wav")

#     def srt_to_dub(self, srt_file_path, dub_save_path, language='en'):
#         srt_result = self.read_srt_file(srt_file_path)
#         new_folder_path = self.create_folder_for_srt(srt_file_path)
#         join_path = []
#         max_accelerate_audio = 2.1
#         update_pause_time = 0

#         # for i in tqdm(range(len(srt_result)), desc="Processing Subtitles"):
#         for i in tqdm_colab(range(len(srt_result)), desc="Processing Subtitles", position=0):
#             text = srt_result[i]['text']
#             actual_duration = srt_result[i]['end_time'] - srt_result[i]['start_time']
#             if i == 0:
#                 pause_time = srt_result[i]['pause_time']
#             else:
#                 pause_time = update_pause_time
#             if i < len(srt_result) - 1:
#                 next_pause_time = srt_result[i + 1]['pause_time']
#             else:
#                 next_pause_time = 0
#             duration_with_next_start = (actual_duration + next_pause_time) / 1000
#             if i == 0:
#                 slient_path = f"{new_folder_path}/{srt_result[i]['previous_pause']}"
#                 self.make_silence(pause_time, slient_path)
#                 join_path.append(slient_path)
#             tts_path = f"{new_folder_path}/{srt_result[i]['audio_name']}"
#             temp_tts_path = self.text_to_speech(text, tts_path, language, actual_duration)
#             tts_audio = AudioSegment.from_file(temp_tts_path)
#             tts_duration = len(tts_audio)
#             if tts_duration == 0:
#                 duration_tts = 0
#             else:
#                 duration_tts = tts_duration / 1000
#             if actual_duration == 0:
#                 duration_true = 0
#             else:
#                 duration_true = actual_duration / 1000
#             acceleration_rate_regulation = False
#             acc_percentage = duration_tts / duration_true
#             # print(f"speed change should be: {acc_percentage}")
#             try:
#                 if acceleration_rate_regulation and acc_percentage >= 1.3:
#                     if duration_with_next_start > duration_true:
#                         extra_time = duration_with_next_start - duration_true
#                         smoth_duration = duration_true + (extra_time * 0.5)
#                         acc_percentage = max(1.2, (duration_tts / smoth_duration))
#             except:
#                 # print("try block failed")
#                 pass
#             if acc_percentage > max_accelerate_audio:
#                 acc_percentage = max_accelerate_audio
#             elif acc_percentage <= 1.15 and acc_percentage >= 0.8:
#                 acc_percentage = 1.0
#             elif acc_percentage <= 0.79:
#                 acc_percentage = 0.8

#             acc_percentage = round(acc_percentage + 0.0, 1)
#             # print(acc_percentage)
#             # print("-------")
#             command = f"ffmpeg -y -loglevel panic -i {temp_tts_path} -filter:a atempo={acc_percentage} {tts_path}"
#             var = os.system(command)
#             if var == 0:
#                 # print("Audio Speed Change")
#                 pass
#             else:
#                 print("Audio Speed Up failed")
#                 print(command)
#             tts_audio = AudioSegment.from_file(tts_path)
#             tts_duration = len(tts_audio)
#             update_pause_time = (duration_with_next_start * 1000) - tts_duration
#             # print(next_pause_time)
#             if i != 0:
#                 slient_path = f"{new_folder_path}/{srt_result[i]['previous_pause']}"
#                 self.make_silence(pause_time, slient_path)
#                 join_path.append(slient_path)
#             join_path.append(tts_path)
#         self.concatenate_audio_files(join_path, dub_save_path)

#     @staticmethod
#     def convert_to_millisecond(time_str):
#         if isinstance(time_str, str):
#             hours, minutes, second_millisecond = time_str.split(':')
#             seconds, milliseconds = second_millisecond.split(",")

#             total_milliseconds = (
#                 int(hours) * 3600000 +
#                 int(minutes) * 60000 +
#                 int(seconds) * 1000 +
#                 int(milliseconds)
#             )

#             return total_milliseconds

#     @staticmethod
#     def read_srt_file(file_path):
#         entries = []
#         default_start = 0
#         previous_end_time = default_start
#         entry_number = 1
#         audio_name_template = "{}.wav"
#         previous_pause_template = "{}_before_pause.wav"

#         with open(file_path, 'r', encoding='utf-8') as file:
#             lines = file.readlines()
#             for i in range(0, len(lines), 4):
#                 time_info = re.findall(r'(\d+:\d+:\d+,\d+) --> (\d+:\d+:\d+,\d+)', lines[i + 1])
#                 start_time = SRTDubbing.convert_to_millisecond(time_info[0][0])
#                 end_time = SRTDubbing.convert_to_millisecond(time_info[0][1])

#                 current_entry = {
#                     'entry_number': entry_number,
#                     'start_time': start_time,
#                     'end_time': end_time,
#                     'text': lines[i + 2].strip(),
#                     'pause_time': start_time - previous_end_time if entry_number != 1 else start_time - default_start,
#                     'audio_name': audio_name_template.format(entry_number),
#                     'previous_pause': previous_pause_template.format(entry_number),
#                 }

#                 entries.append(current_entry)
#                 previous_end_time = end_time
#                 entry_number += 1

#         return entries



# dub_save_path=get_subtitle_Dub_path(srt_file_path,Language)
# #call
# srt_dubbing = SRTDubbing()
# current_language = Language
# srt_dubbing.srt_to_dub(srt_file_path, dub_save_path, current_language)

# # from IPython.display import clear_output
# # clear_output()
# print(f"{Language} Dub Audio File Save At : {dub_save_path}")
# if os.path.exists("/content/gdrive/MyDrive/upload"):
#   file_name = os.path.basename(dub_save_path)
#   shutil.copy(dub_save_path, f"/content/gdrive/MyDrive/upload/{file_name}")
#   print(f"Copied at drive '/content/gdrive/MyDrive/upload/{file_name}'")

# from google.colab import files

# files.download(dub_save_path)
